# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании «N.». Будет разработан такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и обоснована корректность его работы. Защитить данные нужно так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Итоги-исследования" data-toc-modified-id="Итоги-исследования-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итоги исследования</a></span></li></ul></div>

## Загрузка данных

In [1]:
# Сразу подготовим инструменты для дальнейшего использования
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [3]:
train, test = train_test_split(df, test_size=.2, random_state=42)

In [4]:
train['Страховые выплаты'].value_counts()

0    3545
1     345
2      92
3      13
4       5
Name: Страховые выплаты, dtype: int64

Так как целевые значения целочисленные, будем также округлять предсказания и считать R2 как по вещественным, так и по целочисленным значениям.

In [5]:
X_train = train[['Пол', 'Возраст', 'Зарплата', 'Члены семьи']].values
y_train = train['Страховые выплаты'].values
X_test = test[['Пол', 'Возраст', 'Зарплата', 'Члены семьи']].values
y_test = test['Страховые выплаты'].values

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)


Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


- $A$ — сгенерированная обратимая матрица

- $w^\prime$ — новый вектор весов после матричного умножения

- $E$ — единичная матрица

Мы хотим найти веткор весов, который бы давал ту же матрицу предсказаний после матричного умножения:

$$
a = XAw^\prime \Rightarrow Xw = XAw^\prime \Rightarrow w = Aw^\prime \Rightarrow A^{-1}w = A^{-1}Aw^\prime = Ew^\prime = w^\prime
$$

Следовательно:
$$
w^\prime = A^{-1}w
$$

В полном виде:
$$
w^\prime = A^{-1}(X^T X)^{-1} X^T y
$$

Следовательно, чтобы предсказания модели не изменились должно выполняться следующее равенство:

$$
A^{-1}(X^T X)^{-1} X^T y = ((XA)^T XA)^{-1} (XA)^T y
$$

Докажем равенство:

$$
((XA)^T XA)^{-1} (XA)^T y = (A^T X^T XA)^{-1} A^T X^T y = (X^T XA)^{-1} (A^T)^{-1} A^T X^T y = (X^T XA)^{-1} E X^T y = A^{-1} (X^T X)^{-1} X^T y
$$

In [6]:
# Обучаем и тестируем линейную регрессию на изначальных признаках
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)
pred_int = np.round(pred)
print(f'R2: {r2_score(y_test, pred)}\nR2 (целочисленные значения) {r2_score(y_test, pred_int)}')

R2: 0.43686949231379935
R2 (целочисленные значения) 0.630367703779892


Фиксируем значения качества линейной регрессии на изначальных признаках. 

In [7]:
# Создаем обратимую матрицу подходящих размеров
mu = np.random.randint(-100, 100)
sigma = np.random.randint(1, 100)
matrix = np.random.default_rng().normal(mu, sigma, size=(X_train.shape[1], X_train.shape[1]))

In [8]:
# Проверяем обратимость матрицы
np.linalg.det(matrix)

12905735.809600193

$\det A ≠ 0 \Rightarrow A^{-1}существует$

In [9]:
# Умножаем матрицу признаков на получившуюся матрицу
new_matrix = X_train @ matrix
new_matrix

array([[6322431.68789427, -742801.0638358 ,   34174.64671691,
         530768.0338056 ],
       [8114583.40101442, -951536.46936067,   45284.09374361,
         680565.82471947],
       [3737827.42209718, -435899.8505652 ,   22769.92609317,
         312489.58295753],
       ...,
       [5951724.09574152, -696946.15415137,   33978.82760739,
         498739.20377438],
       [4595075.40838937, -537658.21061731,   26565.47534846,
         384907.93809071],
       [5670396.12630465, -663044.57126874,   33138.48620249,
         474826.06326029]])

Очевидно, после перемножения матриц веса в линейной регрессии будут совершенно другими:

In [10]:
model = LinearRegression()
model.fit(new_matrix, y_train)
pred = model.predict(X_test)
pred_int = np.round(pred)
print(f'R2: {r2_score(y_test, pred)}\nR2 (целочисленные значения) {r2_score(y_test, pred_int)}')

R2: -9716.9864628833
R2 (целочисленные значения) -9720.815537413217


Соответственно, тестовые и в будущем новые данные также необходимо умножать на ту же обратимую матрицу, в таком случае качество модели не изменится:

In [11]:
model = LinearRegression()
model.fit(new_matrix, y_train)
pred = model.predict(X_test @ matrix)
pred_int = np.round(pred)
print(f'R2: {r2_score(y_test, pred)}\nR2 (целочисленные значения) {r2_score(y_test, pred_int)}')

R2: 0.4368694923138744
R2 (целочисленные значения) 0.630367703779892


Однако, в том случае, если есть необходимость в раскрытии сгенерированной обратимой матрицы, например, для обработки новых данных перед предсказанием модели, изначальные данные очень легко восстановить, пользуясь свойством обратимости матрицы:

In [12]:
# Получаем обратную матрицу
inv_matrix = np.linalg.inv(matrix)

In [13]:
# Восстанавливаем изначальную матрицу признаков
(np.abs(np.round((new_matrix @ inv_matrix))) == X_train).mean()

0.9979375

Как видно, практически идеальное восстановление данных.

## Алгоритм преобразования

Решить данную проблему восстановления, используя то же матричное умножение, можно взяв необратимую матрицу - любую неквадратную матрицу, фиксируя размерность только одной стороны:

In [14]:
# Создаем необратимую матрицу подходящих размеров
mu = np.random.randint(-100, 100)
sigma = np.random.randint(1, 100)
matrix = np.random.default_rng().normal(mu, sigma, size=(X_train.shape[1], X_train.shape[1] + 10))

In [15]:
# Умножаем матрицу признаков на получившуюся матрицу
new_matrix = X_train @ matrix
new_matrix

array([[ 8841212.67331346,  -177783.1490866 ,  -994536.40800865, ...,
          299344.1310107 ,  2172596.96973789,  5217923.87836732],
       [11345999.51760215,  -227635.41341202, -1274727.39307051, ...,
          384546.05894668,  2788608.69839315,  6696156.53531681],
       [ 5224634.45029101,  -104171.29898963,  -584785.290852  , ...,
          177599.31775413,  1284878.63099547,  3083108.7275051 ],
       ...,
       [ 8321188.98868855,  -166689.07760827,  -933921.08653733, ...,
          282174.68770472,  2045411.47810764,  4910802.69091067],
       [ 6424123.36036645,  -128566.18553142,  -720648.80331942, ...,
          217942.31923566,  1579218.74897855,  3791235.01662331],
       [ 7927134.27318509,  -158530.03631286,  -888981.91715835, ...,
          269141.43462758,  1948992.30191492,  4678193.54984377]])

In [16]:
model = LinearRegression()
model.fit(new_matrix, y_train)
pred = model.predict(X_test @ matrix)
pred_int = np.round(pred)
print(f'R2: {r2_score(y_test, pred)}\nR2 (целочисленные значения) {r2_score(y_test, pred_int)}')

R2: 0.43688180897916384
R2 (целочисленные значения) 0.630367703779892


Как видно, качество моедли фактически не изменяется, а построить обратную матрицу для неквадратной матрицы невозможно. 

Однако существуют способы создания псевдообратной матрицы, с помощью которой можно также восстановить данные:

In [17]:
# Получаем псевдообратную матрицу
pinv_matrix = np.linalg.pinv(matrix)

In [18]:
# Восстанавливаем изначальную матрицу признаков
(np.abs(np.round((new_matrix @ pinv_matrix))) == X_train).mean()

0.9979375

Для решения данной проблемы будет правильнее уменшить колличество признаков, взяв неквадратную матрицу (кол-во признаков на кол-во признаков - n), в таком случае даже построив псевдообратную матрицу восстановить данные не получится. Минус данного подхода в том, что мы потеряем некоторую долю дисперсии, а вместе с ней и часть информации, соответственно качество модели может ухудшиться.

## Проверка алгоритма

In [19]:
# Создаем необратимую матрицу подходящих размеров
mu = np.random.randint(-100, 100)
sigma = np.random.randint(1, 100)
matrix = np.random.default_rng().normal(mu, sigma, size=(X_train.shape[1], X_train.shape[1] - 2))

In [20]:
# Умножаем матрицу признаков на получившуюся матрицу
new_matrix = X_train @ matrix
new_matrix

array([[5801267.54089726, 1661096.5102755 ],
       [7446414.36202145, 2135407.14199688],
       [3431192.55185685,  988695.1287755 ],
       ...,
       [5462002.42529818, 1568179.66411249],
       [4217156.75692784, 1211531.44079138],
       [5204473.33219921, 1496217.47186454]])

In [21]:
model = LinearRegression()
model.fit(new_matrix, y_train)
pred = model.predict(X_test @ matrix)
pred_int = np.round(pred)
print(f'R2: {r2_score(y_test, pred)}\nR2 (целочисленные значения) {r2_score(y_test, pred_int)}')

R2: 0.43266045860638236
R2 (целочисленные значения) 0.6183144767292363


Как и ожидалось, качество модели снизилось, если еще снизить размерность генерируемой матрицы, то качество упадет еще сильнее.

Однако теперь восстановить данные с помощью псевдообратной матрицы невозможно:

In [22]:
# Получаем псевдообратную матрицу
pinv_matrix = np.linalg.pinv(matrix)

In [23]:
# Пробуем восстановить изначальную матрицу признаков
(np.abs(np.round((new_matrix @ pinv_matrix))) == X_train).mean()

0.0

## Итоги исследования

Умножение матрицы признаков на обратимую матрицу не изменяет качетсво модели, но если есть необходимость в раскрытии сгенерированной обратимой матрицы, изначальные данные легко восстановить. Для решения этой проблемы можно генерировать и умножать на необратимую неквадратную матрицу меньшей размерности, однако в таком случае (особенно при небольшом количестве признаков) качество модели может снижаться.